# Teaching Template: Using New AI Models

- Outcomes: Learn setup, make a minimal call, understand capabilities/limits, add an evaluation, and track cost.
- Audience: Practitioners new to this model family.
- Time: ~30–60 minutes.

Prerequisites:
- Python 3.10+; optional GPU for Unsloth/HF fine-tuning.
- Environment variables: `OPENAI_API_KEY` and/or `ANTHROPIC_API_KEY` if using APIs.
- Costs: API examples may incur small charges.

> Tip: Run this notebook top-to-bottom on Colab or your environment.


## Setup
- Prints environment info
- Installs optional packages if missing
- Sets seeds for reproducibility


In [ ]:
import os, sys, platform, random, time
import subprocess

print('Python:', sys.version)
print('Platform:', platform.platform())

def sh(cmd):
    try:
        return subprocess.check_output(cmd, shell=True, text=True).strip()
    except Exception as e:
        return f'ERR: {e}'

print('pip openai:', sh('python -m pip show openai | sed -n "s/^Version: //p"'))
print('pip anthropic:', sh('python -m pip show anthropic | sed -n "s/^Version: //p"'))
print('pip transformers:', sh('python -m pip show transformers | sed -n "s/^Version: //p"'))
print('pip unsloth:', sh('python -m pip show unsloth | sed -n "s/^Version: //p"'))

# Optional: seed setting
import numpy as np
SEED = 42
random.seed(SEED); np.random.seed(SEED)
try:
    import torch
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(SEED)
        print('CUDA device:', torch.cuda.get_device_name(0))
    else:
        print('CUDA not available')
except Exception as e:
    print('Torch not installed; skipping torch seed.', e)


## Secrets & Configuration
Provide API keys via environment variables. This notebook reads them if present and skips API calls if missing.


In [ ]:
from os import getenv
OPENAI_API_KEY = getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = getenv('ANTHROPIC_API_KEY')

def have(pkg):
    try:
        __import__(pkg)
        return True
    except Exception:
        return False

print('Have openai sdk:', have('openai'), 'Have key:', bool(OPENAI_API_KEY))
print('Have anthropic sdk:', have('anthropic'), 'Have key:', bool(ANTHROPIC_API_KEY))


## Quickstart: Minimal Calls
Below are minimal examples for OpenAI and Anthropic. These cells gracefully no-op if SDK or keys are missing.


In [ ]:
# OpenAI minimal chat completion
try:
    if have('openai') and OPENAI_API_KEY:
        from openai import OpenAI
        client = OpenAI(api_key=OPENAI_API_KEY)
        resp = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[{'role':'user','content':'Say hello in 5 words.'}],
            temperature=0
        )
        print(resp.choices[0].message.content)
        usage = resp.usage
        if usage:
            print('Tokens total:', usage.total_tokens)
    else:
        print('OpenAI SDK or key missing; skipping.')
except Exception as e:
    print('OpenAI call error:', e)


In [ ]:
# Anthropic minimal message
try:
    if have('anthropic') and ANTHROPIC_API_KEY:
        import anthropic
        ac = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
        msg = ac.messages.create(
            model='claude-3-5-sonnet-latest',
            max_tokens=64,
            messages=[{'role':'user','content':'Say hello in 5 words.'}]
        )
        print(msg.content[0].text)
    else:
        print('Anthropic SDK or key missing; skipping.')
except Exception as e:
    print('Anthropic call error:', e)


## Model Capabilities & Limits
Briefly summarize what the model is good at, known constraints (context, rate limits), and recommended parameters.


## Guided Steps
1. Define the task
2. Design the prompt or schema
3. Add a tool (if needed)
4. Run a small batch
5. Evaluate results and iterate


In [ ]:
# Example: structured output validation with pydantic (optional)
try:
    from pydantic import BaseModel
    class Item(BaseModel):
        title: str
        rating: int
    print('Pydantic available; define schemas for JSON outputs.')
except Exception:
    print('Pydantic not installed; skip schema validation example.')


## Evaluation
Add a tiny golden set or a simple acceptance test to verify success.


In [ ]:
# Simple acceptance test example
def is_hello_five_words(text: str) -> bool:
    return len(text.split()) == 5

print('Example check:', is_hello_five_words('Hello from a friendly bot'))


## Cost & Observability
Show how to access token usage/cost where supported.


In [ ]:
# Example: OpenAI usage (if available on the response)
# See quickstart cell above for usage fields.
pass


In [ ]:
# Example: Anthropic admin usage/cost API (requires proper entitlements)
print('Refer to Anthropic usage & cost API notebooks for end-to-end examples.')


## Troubleshooting
- 401/403: check API keys and org permissions
- 429: add exponential backoff; reduce concurrency; shorten prompts
- JSON errors: validate with schemas; enable JSON/structured modes if available
- Timeouts: reduce payloads; increase timeout; retry idempotently


## Exercises
- Change the prompt to require a JSON schema and validate it
- Add a tool/function call and parse its output
- Create a 20-item golden set and compute accuracy
- Measure average latency/tokens over 10 runs


## Cleanup
- Stop any background jobs or servers
- Clear temp files or caches if created
- Save artifacts/models if you trained locally
